In [2]:
# Import required libraries for API requests and YAML config parsing
import requests
import json
import yaml
from pathlib import Path
# Import the main extractor class from the installed package
from monday_board_extractor import MondayColumnExtractor

In [3]:
import json
import logging
import requests
from typing import Any


import requests
import json
import logging

class MondayColumnUpdater:
    def __init__(self, api_key: str):
        self.url = "https://api.monday.com/v2"
        self.headers = {
            "Authorization": api_key,
            "Content-Type": "application/json"
        }

    def update_column_value(
        self,
        board_id: int,
        item_id: int,
        column_id: str,
        value: str
        ) -> dict:
            # Double encoding required by Monday.com GraphQL spec
        wrapped_value = json.dumps(json.dumps({"text": value}))

        mutation = f'''
            mutation {{
                change_column_value(
                    board_id: {board_id},
                    item_id: {item_id},
                    column_id: "{column_id}",
                    value: {wrapped_value}
                ) {{
                id
                }}
            }}
        '''

        payload = {"query": mutation}
        print("=== GraphQL Mutation Payload ===")
        print(json.dumps(payload, indent=2))

        try:
            response = requests.post(
                self.url,
                json=payload,
                headers=self.headers
            )
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            logging.error(f"Update request failed: {e}")
            raise
        except Exception as e:
            logging.error(f"Unexpected error during update: {e}")
            raise



In [4]:
# Automatically look for 'monday_config.yaml' in the current working directory (project root)
config_path = Path.cwd() / "monday_config.yaml"
if not config_path.exists():
    raise FileNotFoundError(f"Config file not found at {config_path}. Please ensure 'monday_config.yaml' is in the project root.")

with config_path.open("r") as f:
    config = yaml.safe_load(f)

api_key = config["monday"]["api_key"]

In [ ]:
board_ids = [8033166260]

extractor = MondayColumnExtractor(api_key, board_ids)
results = extractor.extract()

# To access one dataframe by board name:
for board_data in results:
    print(board_data.name)

board_data.data

PD Program


,pm_Mjj4IJZX,financials_Mjj4L6Yo,dor_Mjj4yLIK,active_task_Mjj4FJQL,comment_Mjj4ErAr,contractor_Mjj47NUD,date_mkv8epda,date_mkv89r9t,date_mkv8xss3,project_timeline,...,formula_mktys6gm,project_duration,project_task_completion_date,date_mktynmfm,formula_mktyrxna,subitems_Mjj4Ab1e,project_owner,project_status,active_task_Mjj45kec,project_budget
0,,,,Planning,,,,,,,...,,,,,,None,,None,,
1,Martin Blancas,"Total Project Budget: $12,000,000.00\nDesign B...",MNK,Design,"Plans sent to PD, EPWater, Facilities, IT, and...",TBD,,,,,...,,,,,,None,,None,,
2,Nancy Lopez,"Total Project Budget: $8,081,821.00\nDesign Bu...",DEKKER,Design,30% CDR scheduled for 8/27/2025,TBD,,,,,...,,,,,,None,,None,,
3,Jesus Palma,"Total Project Budget: $7,105,706\nDesign Budge...",CDA/DEKKER,Predesign,Restrooms to be completed by 10/20/2025. Docu...,VELIZ,,,,,...,,,,,,None,,None,,
4,Jorge Chacon,"Total Project Budget: $7,430,635.47\nDesign Bu...",TBD,Predesign,In negotiations with MNK on proposal. Rate fo...,TBD,,,,,...,,,,,,None,,None,,
5,Daniel Carrillo,"Total Project Budget: $23,616,000\nDesign Budg...",ASA,Design,60% plans due 9/26/2025. Last OPC provided wa...,TBD,,,,,...,,,,,,None,,None,,
6,Nicholas Gonzales,Total Project Budget: $\nDesign Budget: $\nCon...,DEKKER,Planning,"Reduce sq ft reduction by 30,000. Pending upd...",TBD,,,,,...,,,,,,None,,None,,


: 

In [18]:
updater = MondayColumnUpdater(api_key=api_key)

new_financial_text = (
    "Total Project Budget: $12,000,000.00\n"
    "Design Budget: $900,000.00\n"
    "Construction Budget: $9,000,000.00"
)

response = updater.update_column_value(
    board_id=8033166260,
    item_id=8034118992,
    column_id="financials_Mjj4L6Yo",
    value=new_financial_text
)

print(response)

=== GraphQL Mutation Payload ===
{
  "query": "\n        mutation {\n        change_column_value(\n        board_id: 8033166260,\n        item_id: 8034118992,\n        column_id: \"financials_Mjj4L6Yo\",\n        value: \"{\\\"text\\\": \\\"Total Project Budget: $12,000,000.00\\\\nDesign Budget: $900,000.00\\\\nConstruction Budget: $9,000,000.00\\\"}\"\n      ) {\n        id\n      }\n    }\n    "
}
{'data': {'change_column_value': {'id': '8034118992'}}, 'extensions': {'request_id': '17dc3d40-7ceb-9548-b7fd-532be4e34c2f'}}


In [19]:
extractor._fetch_data()

{'data': {'boards': [{'name': 'PD Program',
    'items_page': {'items': [{'id': '9842734550',
       'column_values': [{'id': 'pm_Mjj4IJZX', 'text': ''},
        {'id': 'financials_Mjj4L6Yo', 'text': ''},
        {'id': 'dor_Mjj4yLIK', 'text': ''},
        {'id': 'active_task_Mjj4FJQL', 'text': 'Planning'},
        {'id': 'comment_Mjj4ErAr', 'text': ''},
        {'id': 'contractor_Mjj47NUD', 'text': ''},
        {'id': 'project_timeline', 'text': ''},
        {'id': 'timerange_mkty93q3', 'text': ''},
        {'id': 'formula_mktys6gm', 'text': ''},
        {'id': 'project_duration', 'text': ''},
        {'id': 'project_task_completion_date', 'text': ''},
        {'id': 'date_mktynmfm', 'text': ''},
        {'id': 'formula_mktyrxna', 'text': ''},
        {'id': 'subitems_Mjj4Ab1e', 'text': None},
        {'id': 'project_owner', 'text': ''},
        {'id': 'project_status', 'text': None},
        {'id': 'active_task_Mjj45kec', 'text': ''},
        {'id': 'project_budget', 'text': ''}]},
 